In [16]:
import pennylane as qml
import numpy as np

from QHyper.optimizers.cem import CEM
from QHyper.optimizers.random import Random
from QHyper.optimizers.qml_gradient_descent import QmlGradientDescent
from QHyper.solvers.qaoa.core import QAOA
from QHyper.solvers.converter import Converter

## TSP problem

In [3]:
from QHyper.problems.tsp import TSPProblem

In [4]:
tsp = TSPProblem(
    number_of_cities=3,
)

In [6]:
print(tsp.objective_function)

+0.6461134730199508*(+x0*x4+x3*x7+x6*x1)+0.36604300091974296*(+x0*x5+x3*x8+x6*x2)+0.6461134730199508*(+x1*x3+x4*x6+x7*x0)+1.0*(+x1*x5+x4*x8+x7*x2)+0.36604300091974296*(+x2*x3+x5*x6+x8*x0)+1.0*(+x2*x4+x5*x7+x8*x1)


In [ ]:
solver = QAOA(
    problem=tsp,
    platform="pennylane",
    optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 200),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # mixer: str=,
    weights=[1, 1, 1],
    # hyperoptimizer=CEM(
    #     epochs=5,
    #     samples_per_epoch=100,
    #     elite_frac=0.1
    # ),
    # backend=
)

In [ ]:
value, params, weights = solver.solve()

In [ ]:
value

In [ ]:
probs = solver.get_probs_func(weights)(params)

In [ ]:
solver.check_results(probs)

In [ ]:
solver.print_results(probs)

## Knapsack problem

In [7]:
from QHyper.problems.knapsack import Knapsack, KnapsackProblem

In [19]:
kp = KnapsackProblem(max_weight=2, items=[(2, 2), (3, 1)])

In [20]:
kp.objective_function.polynomial

-2*x0 - x1

In [22]:
for constraint in kp.constraints:
    print(constraint.polynomial)

(-x2 - x3 + 1)**2
(-2*x0 - 3*x1 + x2 + 2*x3)**2


In [24]:
converter = Converter()
kp_cqm = converter.to_cqm(kp)
# kp_qubo = converter.to_qubo(kp)

In [25]:
print(kp_cqm)

Constrained quadratic model: 4 variables, 2 constraints, 15 biases

Objective
  -2*Binary('x0') - Binary('x1')

Constraints
  cc0831d: 1 - Binary('x2') - Binary('x3') + 2*Binary('x2')*Binary('x3') == 0.0
  c54867f: 4*Binary('x0') + 9*Binary('x1') + Binary('x2') + 4*Binary('x3') + 12*Binary('x0')*Binary('x1') - 4*Binary('x0')*Binary('x2') - 6*Binary('x1')*Binary('x2') - 8*Binary('x0')*Binary('x3') - 12*Binary('x1')*Binary('x3') + 4*Binary('x2')*Binary('x3') == 0.0

Bounds



In [27]:
import dimod

In [28]:
bqm, _ = dimod.cqm_to_bqm(kp_cqm)

ValueError: CQM must not have any quadratic constraints

In [ ]:
solver = QAOA(
    problem=knapsack_qaoa,
    platform="pennylane",
    optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 200),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # weights=[2.72616422, 4.59436366, 4.46463072],
    weights=[1, 1, 1]
    hyperoptimizer=CEM(),
    # backend=
)
value, params, weights = solver.solve()

In [ ]:
value

In [ ]:
probs = solver.get_probs_func(weights)(params)

In [ ]:
solver.check_results(probs)

In [ ]:
solver.print_results(probs)

## Workflow Scheduling Problem

In [29]:
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingProblem
from QHyper.solvers.converter import Converter

In [30]:
tasks_file = "/Users/jzawalska/Coding/QHyper/QHyper/problems/workflows/workflows_data/workflows/3_tasks_3_machines_1_path.json"
machines_file = "/Users/jzawalska/Coding/QHyper/QHyper/problems/workflows/workflows_data/machines/machines_for_3_tasks_3_machines_1_path.json"
deadline = 20

In [31]:
workflow = Workflow(tasks_file, machines_file, deadline)
wsp = WorkflowSchedulingProblem(workflow)

In [34]:
converter = Converter()
wsp_qubo, offset = converter.to_qubo(wsp)

In [35]:
print(wsp_qubo)

{('x1', 'x0'): 260.0, ('x2', 'x0'): 500.0, ('x2', 'x1'): 180.0, ('x3', 'x0'): 360.0, ('x3', 'x1'): 120.0, ('x3', 'x2'): 240.0, ('x4', 'x0'): 120.0, ('x4', 'x1'): 40.0, ('x4', 'x2'): 80.0, ('x4', 'x3'): 80.0, ('x5', 'x0'): 240.0, ('x5', 'x1'): 80.0, ('x5', 'x2'): 160.0, ('x5', 'x3'): 140.0, ('x5', 'x4'): 60.0, ('x6', 'x0'): 1440.0, ('x6', 'x1'): 480.0, ('x6', 'x2'): 960.0, ('x6', 'x3'): 720.0, ('x6', 'x4'): 240.0, ('x6', 'x5'): 480.0, ('x7', 'x0'): 480.0, ('x7', 'x1'): 160.0, ('x7', 'x2'): 320.0, ('x7', 'x3'): 240.0, ('x7', 'x4'): 80.0, ('x7', 'x5'): 160.0, ('x7', 'x6'): 980.0, ('x8', 'x0'): 960.0, ('x8', 'x1'): 320.0, ('x8', 'x2'): 640.0, ('x8', 'x3'): 480.0, ('x8', 'x4'): 160.0, ('x8', 'x5'): 320.0, ('x8', 'x6'): 1940.0, ('x8', 'x7'): 660.0, ('s0', 'x0'): 120.0, ('s0', 'x1'): 40.0, ('s0', 'x2'): 80.0, ('s0', 'x3'): 60.0, ('s0', 'x4'): 20.0, ('s0', 'x5'): 40.0, ('s0', 'x6'): 240.0, ('s0', 'x7'): 80.0, ('s0', 'x8'): 160.0, ('s1', 'x0'): 240.0, ('s1', 'x1'): 80.0, ('s1', 'x2'): 160.0, ('

In [38]:
solver = QAOA(
    problem=wsp,
    platform="pennylane",
    optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 200),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # weights=[2.72616422, 4.59436366, 4.46463072],
    weights=[1],
    hyperoptimizer=CEM(),
    # backend=
)
value, params, weights = solver.solve()

  0%|                                                   | 0/101 [00:02<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'items'